In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from xgboost import XGBRegressor

Subir el token de kaggle para descargar los archivos del dataset:

In [ ]:
! pip install -q kaggle

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"simonnarvaezrestrepo","key":"ad4aafdf127ae87f181179ced16aecc6"}'}

Descargar el data set:

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c playground-series-s3e20
! unzip playground-series-s3e20.zip -d playground-series-s3e20

 57% 28.0M/48.9M [00:00<00:00, 155MB/s] 
100% 48.9M/48.9M [00:00<00:00, 182MB/s]
Archive:  playground-series-s3e20.zip
  inflating: playground-series-s3e20/sample_submission.csv  
  inflating: playground-series-s3e20/test.csv  
  inflating: playground-series-s3e20/train.csv  


Read the dataset with:

In [ ]:
%matplotlib inline
d = pd.read_csv("playground-series-s3e20/train.csv")
d.head()
t = pd.read_csv("playground-series-s3e20/test.csv")
t.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
0,ID_-0.510_29.290_2022_00,-0.51,29.29,2022,0,NaN,NaN,NaN,NaN,NaN,...,36022.027344,8472.313477,41047.937500,7472.313477,7.935617,0.240773,-100.113792,33.697044,-133.047546,33.779583
1,ID_-0.510_29.290_2022_01,-0.51,29.29,2022,1,0.000456,0.691164,0.000316,0.000000,76.239196,...,48539.737242,6476.147323,54915.708579,5476.147161,11.448437,0.293119,-30.510319,42.402593,-138.632822,31.012380
2,ID_-0.510_29.290_2022_02,-0.51,29.29,2022,2,0.000161,0.605107,0.000106,0.079870,-42.055341,...,34133.080469,8984.795703,39006.093750,7984.795703,10.753179,0.267130,39.087361,45.936480,-144.784988,26.743361
3,ID_-0.510_29.290_2022_03,-0.51,29.29,2022,3,0.000350,0.696917,0.000243,0.201028,72.169566,...,50854.991076,6014.724059,57646.368368,5014.724115,11.764556,0.304679,-24.465127,42.140419,-135.027891,29.604774
4,ID_-0.510_29.290_2022_04,-0.51,29.29,2022,4,-0.000317,0.580527,-0.000184,0.204352,76.190865,...,46594.685145,6849.280477,52896.541873,5849.280394,13.065317,0.284221,-12.907850,30.122641,-135.500119,26.276807


In [ ]:
print (d.shape)

(79023, 76)


In [ ]:
k = d.isna().sum()
k[k!=0]

SulphurDioxide_SO2_column_number_density          14609
SulphurDioxide_SO2_column_number_density_amf      14609
SulphurDioxide_SO2_slant_column_number_density    14609
SulphurDioxide_cloud_fraction                     14609
SulphurDioxide_sensor_azimuth_angle               14609
                                                  ...  
Cloud_surface_albedo                                484
Cloud_sensor_azimuth_angle                          484
Cloud_sensor_zenith_angle                           484
Cloud_solar_azimuth_angle                           484
Cloud_solar_zenith_angle                            484
Length: 70, dtype: int64

In [ ]:
d.columns

Index(['ID_LAT_LON_YEAR_WEEK', 'latitude', 'longitude', 'year', 'week_no',
       'SulphurDioxide_SO2_column_number_density',
       'SulphurDioxide_SO2_column_number_density_amf',
       'SulphurDioxide_SO2_slant_column_number_density',
       'SulphurDioxide_cloud_fraction', 'SulphurDioxide_sensor_azimuth_angle',
       'SulphurDioxide_sensor_zenith_angle',
       'SulphurDioxide_solar_azimuth_angle',
       'SulphurDioxide_solar_zenith_angle',
       'SulphurDioxide_SO2_column_number_density_15km',
       'CarbonMonoxide_CO_column_number_density',
       'CarbonMonoxide_H2O_column_number_density',
       'CarbonMonoxide_cloud_height', 'CarbonMonoxide_sensor_altitude',
       'CarbonMonoxide_sensor_azimuth_angle',
       'CarbonMonoxide_sensor_zenith_angle',
       'CarbonMonoxide_solar_azimuth_angle',
       'CarbonMonoxide_solar_zenith_angle',
       'NitrogenDioxide_NO2_column_number_density',
       'NitrogenDioxide_tropospheric_NO2_column_number_density',
       'NitrogenDioxide

In [ ]:
ccols = [i for i in d.columns if not i in d._get_numeric_data()]
print (ccols)

['ID_LAT_LON_YEAR_WEEK']


In [ ]:
for c in ccols:
    print ("%10s"%c, np.unique(d[c].dropna()))

ID_LAT_LON_YEAR_WEEK ['ID_-0.510_29.290_2019_00' 'ID_-0.510_29.290_2019_01'
 'ID_-0.510_29.290_2019_02' ... 'ID_-3.299_30.301_2021_50'
 'ID_-3.299_30.301_2021_51' 'ID_-3.299_30.301_2021_52']


Se imprime el porcentaje de valores faltantes en cada columna

In [ ]:
(d.isnull().sum() * 100/d.shape[0]).sort_values(ascending = False)

UvAerosolLayerHeight_aerosol_height          99.444466
UvAerosolLayerHeight_solar_zenith_angle      99.444466
UvAerosolLayerHeight_solar_azimuth_angle     99.444466
UvAerosolLayerHeight_sensor_azimuth_angle    99.444466
UvAerosolLayerHeight_aerosol_pressure        99.444466
                                               ...    
latitude                                      0.000000
week_no                                       0.000000
year                                          0.000000
longitude                                     0.000000
emission                                      0.000000
Length: 76, dtype: float64

# Limpieza
Se planean dos estrategias de limpieza, una donde eliminaremos las columnas con un porcetanje de 99.4% de datos faltantes y rellenando los datos faltantes con la media de los datos. Y la segunda es donde eliminaremos las columnas por este mismo criterio, pero en esta ocasión se eliminaran las filas con datos faltantes

Ya que se identifica, que hay columnas con un valor de 99.4%, lo cuál representa un alto porcentaje de datos faltantes, tomamos la decisión de eliminar dichas columnas, como primer proceso de limpieza

In [ ]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 76 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   ID_LAT_LON_YEAR_WEEK                                      79023 non-null  object 
 1   latitude                                                  79023 non-null  float64
 2   longitude                                                 79023 non-null  float64
 3   year                                                      79023 non-null  int64  
 4   week_no                                                   79023 non-null  int64  
 5   SulphurDioxide_SO2_column_number_density                  64414 non-null  float64
 6   SulphurDioxide_SO2_column_number_density_amf              64414 non-null  float64
 7   SulphurDioxide_SO2_slant_column_number_density            64414 non-null  float64
 8   SulphurDioxide_c

In [ ]:
newData = d.drop(d.columns[57:64], axis=1)

Aqui, el siguiente proceso para limpieza de datos, llenan los datos vacios con la media de los datos

In [ ]:
from sklearn.impute import SimpleImputer

imp_median = SimpleImputer(missing_values=np.nan, strategy='median')

In [ ]:
df_clean = imp_median.fit_transform(newData.iloc[:, 1:-1])

In [ ]:
df_impute = pd.DataFrame(data = df_clean, columns = newData.columns[1:-1])

Quedamos con un dataframe que no cuenta con las columnas con un porcentaje de 99.4% de datos vacíos y remplazando los datos faltantes con la media, siendo este el dataframe df_impute

In [ ]:
df_impute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 67 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   latitude                                                  79023 non-null  float64
 1   longitude                                                 79023 non-null  float64
 2   year                                                      79023 non-null  float64
 3   week_no                                                   79023 non-null  float64
 4   SulphurDioxide_SO2_column_number_density                  79023 non-null  float64
 5   SulphurDioxide_SO2_column_number_density_amf              79023 non-null  float64
 6   SulphurDioxide_SO2_slant_column_number_density            79023 non-null  float64
 7   SulphurDioxide_cloud_fraction                             79023 non-null  float64
 8   SulphurDioxide_s

Se hace la asignación con los datos procesados en dos variables, X, y Y, donde X es el dataFrame que contiene las características con los valores faltantes imputados. y Y contiene la columna emission del dataFrame sin las columnas previamente eliminadas. Y contiene la columna emission que representa la variable objetivo o la que el modelo a implementar intentará predecir.

In [ ]:
X = df_impute
Y = newData['emission']

A continuación creamos una segunda versión del dataframe en la cual eliminamos las filas que contienen datos vacios, dado que eran más de 20.000 filas con datos faltantes queriamos comprobar cual de las dos versiones tenía mejores resultados

In [ ]:
df_row_clean = newData.dropna(0)

In [ ]:
df_row_clean.info()

Se hace la asignación de la segunda versión del dataframe a X_RC, analogo a X

In [ ]:
X_RC = df_row_clean.drop(['ID_LAT_LON_YEAR_WEEK', 'emission'], axis = 1).fillna(0)

In [ ]:
X_RC

,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
1,-0.510,29.290,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,39.137194,...,63790.296241,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779
2,-0.510,29.290,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,52.868816,...,55923.790554,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296
4,-0.510,29.290,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,35.515587,...,59904.314844,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612
5,-0.510,29.290,2019,5,0.000294,0.871713,0.000242,0.227656,-13.453690,57.097124,...,43494.921743,5530.354546,49846.001010,4495.301362,7.556143,0.250228,30.128401,37.007020,-137.388719,21.157820
6,-0.510,29.290,2019,6,-0.000285,0.791956,-0.000226,0.119397,72.198647,58.496368,...,53483.783984,4378.614648,58804.276563,3537.050659,10.493107,0.240828,4.585368,30.491541,-128.196338,21.721202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79017,-3.299,30.301,2021,47,-0.000257,0.668691,-0.000172,0.257957,71.891731,65.816658,...,47198.775641,6894.667968,53218.918203,5894.667968,25.124038,0.308299,24.648991,38.736509,-129.933645,29.499534
79018,-3.299,30.301,2021,48,0.000284,1.195643,0.000340,0.191313,72.820518,55.988022,...,54703.511452,5459.185355,60657.101913,4590.879504,20.245954,0.304797,-35.140368,40.113533,-129.935508,32.095214
79019,-3.299,30.301,2021,49,0.000083,1.130868,0.000063,0.177222,-12.856753,19.435339,...,53983.493958,5606.449457,60168.191528,4659.130378,6.104610,0.314015,4.667058,47.528435,-134.252871,30.771469
79021,-3.299,30.301,2021,51,-0.000034,0.879397,-0.000028,0.184209,-100.344827,32.599393,...,41395.313100,7896.456885,46533.348194,6946.858022,32.594768,0.274047,8.427699,48.295652,-139.447849,29.112868


In [ ]:
Y_RC = df_row_clean['emission']

In [ ]:
Y_RC

1         4.025176
2         4.231381
4         4.347317
5         4.310819
6         4.269334
           ...    
79017    30.297634
79018    29.404171
79019    29.186497
79021    28.125792
79022    27.239302
Name: emission, Length: 57209, dtype: float64

# Modelos


Random Forest

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=45)

In [ ]:
clf = RandomForestRegressor(random_state = 45, n_jobs=-1)

In [ ]:
clf.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=45)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(f'RMSE Score: {mean_squared_error(y_test, y_pred, squared=False)}')
print("R^2 : ", r2_score(y_test, y_pred))
print("MAE :", mean_absolute_error(y_test,y_pred))


RMSE Score: 22.535370367758908
R^2 :  0.9747592356050864
MAE : 8.773575390916621


XGBC

In [ ]:
X_RC_train, X_RC_test, Y_RC_train, Y_RC_test = train_test_split(X_RC, Y_RC, test_size = 0.3, random_state = 42)


In [ ]:
xgb = XGBRegressor()

In [ ]:
xgb.fit(X_RC_train, Y_RC_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_rc_pred = xgb.predict(X_RC_test)

In [ ]:
print(f'RMSE Score: {mean_squared_error(Y_RC_test, y_rc_pred, squared=False)}')
print("R^2 : ", r2_score(Y_RC_test, y_rc_pred))
print("MAE :", mean_absolute_error(Y_RC_test,y_rc_pred))

RMSE Score: 25.709075973684286
R^2 :  0.9657398312435168
MAE : 13.879364612260398
